In [26]:
%load_ext autoreload
%autoreload 2

import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
from torch.utils.data import TensorDataset, DataLoader
from tqdm import tqdm

from PhaseFunctionedNetwork import PhaseFunctionedNetwork

# set seeds for reproduceability
torch.manual_seed(42)
np.random.seed(42)
rng = np.random.RandomState(42)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [27]:
# Process Data Mirrored

# Idle Data
X_idle = np.float32(np.loadtxt('C:/Users/Ana/Desktop/dev/pfnn-dev/Export/Input_Idle_Static.txt'))
Y_idle = np.float32(np.loadtxt('C:/Users/Ana/Desktop/dev/pfnn-dev/Export/Output_Idle_Static.txt'))
P_idle = np.linspace(0, 2*np.pi, num=72)
print(f"Idle data shape: {X_idle.shape}, {Y_idle.shape}, {P_idle.shape}")

# Jump Flat
X_jump_horizflat = np.float32(np.loadtxt('C:/Users/Ana/Desktop/dev/pfnn-dev/Export/Input_Jump_HorizFlat.txt'))
Y_jump_horizflat = np.float32(np.loadtxt('C:/Users/Ana/Desktop/dev/pfnn-dev/Export/Output_Jump_HorizFlat.txt'))
jump_horizflat_1 = np.linspace(0, 2*np.pi, num=50)
jump_horizflat_2 = np.linspace(0, 2*np.pi, num=50)
P_jump_horizflat = np.concatenate((jump_horizflat_1, jump_horizflat_2))
print(f"Idle data shape: {X_jump_horizflat.shape}, {Y_jump_horizflat.shape}, {P_jump_horizflat.shape}")

# Jump Up
X_jump_horizup = np.float32(np.loadtxt('C:/Users/Ana/Desktop/dev/pfnn-dev/Export/Input_Jump_HorizUp.txt'))
Y_jump_horizup = np.float32(np.loadtxt('C:/Users/Ana/Desktop/dev/pfnn-dev/Export/Output_Jump_HorizUp.txt'))
jump_horizup_1 = np.linspace(0, 2*np.pi, num=48)
jump_horizup_2 = np.linspace(0, 2*np.pi, num=48)
P_jump_horizup = np.concatenate((jump_horizup_1, jump_horizup_2))
print(f"Idle data shape: {X_jump_horizup.shape}, {Y_jump_horizup.shape}, {P_jump_horizup.shape}")


Idle data shape: (72, 1836), (72, 1767), (72,)
Idle data shape: (100, 1836), (100, 1767), (100,)
Idle data shape: (96, 1836), (96, 1767), (96,)


In [28]:
# Concatenate Data into indivudal X, Y, P Arrays
X = np.concatenate((X_idle, X_jump_horizflat, X_jump_horizup))
print(f"X: {X.shape}")

Y = np.concatenate((Y_idle, Y_jump_horizflat, Y_jump_horizup))
print(f"Y: {Y.shape}")

P = np.concatenate((P_idle, P_jump_horizflat, P_jump_horizup))
print(f"P: {P.shape}")

X: (268, 1836)
Y: (268, 1767)
P: (268,)


In [29]:
# Phase logic
delta_phase = P[1:] - P[:-1]
delta_phase[delta_phase < 0] = (1.0 - P[:-1] + P[1:])[delta_phase < 0]
delta_phase = np.append(delta_phase, 2 * np.pi)

Y = np.concatenate([Y, delta_phase [..., np.newaxis]], axis=-1)

# calculate mean and std
Xmean, Xstd = X.mean(axis=0), X.std(axis=0)
Ymean, Ystd = Y.mean(axis=0), Y.std(axis=0)

for i in range(Xstd.size):
    if (Xstd[i]==0):
        Xstd[i]=1
for i in range(Ystd.size):
    if (Ystd[i]==0):
        Ystd[i]=1

# normalize data
X = (X - Xmean) / Xstd
Y = (Y - Ymean) / Ystd

# save mean / std / min / max

Xmean.astype(np.float32).tofile('C:/Users/Ana/Desktop/dev/pfnn-dev/unity-pfnn/Assets/Dev/Weights/test/Xmean.bin')
Ymean.astype(np.float32).tofile('C:/Users/Ana/Desktop/dev/pfnn-dev/unity-pfnn/Assets/Dev/Weights/test/Ymean.bin')
Xstd.astype(np.float32).tofile('C:/Users/Ana/Desktop/dev/pfnn-dev/unity-pfnn/Assets/Dev/Weights/test/Xstd.bin')
Ystd.astype(np.float32).tofile('C:/Users/Ana/Desktop/dev/pfnn-dev/unity-pfnn/Assets/Dev/Weights/test/Ystd.bin')

In [30]:
# append phase as additional feature
input = torch.tensor(np.concatenate([X, P [..., np.newaxis]], axis=-1))
target = torch.tensor(Y)

print(f"Input shape {input.shape}")
print(f"Target shape {target.shape}")

dataset = TensorDataset(input, target)

BATCH_SIZE = 32
train_dataloader = DataLoader(dataset, batch_size=BATCH_SIZE, shuffle=True)

Input shape torch.Size([268, 1837])
Target shape torch.Size([268, 1768])


In [31]:
# ensure GPU is available
# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# print(f"Using device: {device}")
device = "cuda"

# custom loss function
def loss_func(output, target, model):
    loss = torch.mean((output - target)**2) + model.cost()
    return loss

In [32]:
model = PhaseFunctionedNetwork(input_shape=input.shape[1], output_shape=target.shape[1])
model.to(device)

optimizer = torch.optim.AdamW(model.parameters(), lr=0.001)

epochs=20

for epoch in range(epochs):
    model.train()
    loss_list = []
    for i, batch in enumerate(tqdm(train_dataloader)):
        input, target = batch
        input, target = input.to(device), target.to(device)

        # forward pass
        output = model(input)
        loss = loss_func(output, target, model)
        loss_list.append(loss.item())

        # backward pass
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    
    print(f'Epoch [{epoch+1}/{epochs}], Loss: {np.average(loss_list)}')


100%|██████████| 9/9 [00:01<00:00,  5.94it/s]


Epoch [1/20], Loss: 0.8700473125268599


100%|██████████| 9/9 [00:01<00:00,  6.73it/s]


Epoch [2/20], Loss: 0.6971103350168839


100%|██████████| 9/9 [00:01<00:00,  6.66it/s]


...

Epoch [18/20], Loss: 0.36116064966739675


100%|██████████| 9/9 [00:01<00:00,  6.98it/s]


Epoch [19/20], Loss: 0.36235240318215756


100%|██████████| 9/9 [00:01<00:00,  6.98it/s]

Epoch [20/20], Loss: 0.35357057195721836


In [33]:
# save weights
model.precompute_and_save_weights()